In [1]:
import cv2
import numpy as np
import dlib
from math import hypot

In [2]:
cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#Keyboard settings
keyboard = np.zeros((600,1000,3),np.uint8)
keys_set_1 = {0: "Q", 1:"W", 2:"E", 3:"R", 4:"T",
              5:"A", 6:"S", 7:"D", 8:"F", 9:"G",
             10:"Z", 11:"X", 12:"C", 13:"V", 14:"B"}
def letter(letter_index,text,letter_light):
    #Keys
    if letter_index == 0:
        x = 0
        y = 0
    elif letter_index == 1:
        x = 200
        y = 0
    elif letter_index == 2:
        x = 400
        y = 0
    elif letter_index == 3:
        x = 600
        y = 0
    elif letter_index == 4:
        x = 800
        y = 0
    elif letter_index == 5:
        x = 0
        y = 200
    elif letter_index == 6:
        x = 200
        y = 200
    elif letter_index == 7:
        x = 400
        y = 200
    elif letter_index == 8:
        x = 600
        y = 200
    elif letter_index == 9:
        x = 800
        y = 200
    elif letter_index == 10:
        x = 0
        y = 400
    elif letter_index == 11:
        x = 200
        y = 400
    elif letter_index == 12:
        x = 400
        y = 400
    elif letter_index == 13:
        x = 600
        y = 400
    elif letter_index == 14:
        x = 800
        y = 400
        
    width = 200
    height = 200
    th = 3 #thickness
    if letter_light is True:
        cv2.rectangle(keyboard,(x+th,y+th),(x+width-th,y+height-th),(255,255,255),-1) #-1 here means fill the rectangle
    else:
        cv2.rectangle(keyboard,(x+th,y+th),(x+width-th,y+height-th),(255,0,0),th)
    

    # Text settings
    font_letter = cv2.FONT_HERSHEY_PLAIN
    text = text
    font_scale = 10
    font_th = 4 #font thinkness
    text_size = cv2.getTextSize(text, font_letter, font_scale, font_th)[0]
    width_text, height_text = text_size[0],text_size[1]

    text_x = int((width-width_text)/2) + x
    text_y= int((height+height_text)/2) + y

    cv2.putText(keyboard, text,(text_x,text_y),font_letter, font_scale, (255,0,0),font_th)

    # print(text_size)



def midpoint(p1,p2):
    return int((p1.x+p2.x)/2) , int((p1.y+p2.y)/2)

font = cv2.FONT_HERSHEY_SIMPLEX

def get_blinking_ratio(eye_points, facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)

#     hor_line = cv2.line(frame, left_point, right_point,(0,255,0), 2)

    center_top = midpoint(facial_landmarks.part(eye_points[1]),facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]),facial_landmarks.part(eye_points[4]))

#     ver_line = cv2.line(frame, center_top, center_bottom ,(0,255,0), 2)

    #Checking length of horizontal and vertical lines
    ver_line_length = hypot((center_top[0]-center_bottom[0]),(center_top[1]-center_bottom[1]))
    hor_line_length = hypot((left_point[0]-right_point[0]),(left_point[1]-right_point[1]))
        
    ratio = hor_line_length/ver_line_length
    return ratio

def get_gaze_ratio(eye_points, facial_landmarks):
    left_eye_region = np.array([(facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y),
                                    (facial_landmarks.part(eye_points[1]).x, facial_landmarks.part(eye_points[1]).y),
                                    (facial_landmarks.part(eye_points[2]).x, facial_landmarks.part(eye_points[2]).y),
                                    (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y),
                                    (facial_landmarks.part(eye_points[4]).x, facial_landmarks.part(eye_points[4]).y),
                                    (facial_landmarks.part(eye_points[5]).x, facial_landmarks.part(eye_points[5]).y)
                                   ],np.int32)
        
#         print(left_eye_region)
#         cv2.polylines(frame, [left_eye_region] , True, (0,0,255),2)

    height, width, _ = frame.shape
    mask = np.zeros((height,width), np.uint8)

    cv2.polylines(mask, [left_eye_region],True, 255,2)
    cv2.fillPoly(mask, [left_eye_region], 255)
    eye = cv2.bitwise_and(gray,gray, mask=mask)        

    min_x = np.min(left_eye_region[:,0])
    max_x = np.max(left_eye_region[:,0])
    min_y = np.min(left_eye_region[:,1])
    max_y = np.max(left_eye_region[:,1])

#         eye = frame[min_y:max_y,min_x:max_x]
#         gray_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
#         _,threshold_eye = cv2.threshold(gray_eye, 40, 255, cv2.THRESH_BINARY)
    gray_eye = eye[min_y:max_y,min_x:max_x]
    _,threshold_eye = cv2.threshold(gray_eye, 40, 255, cv2.THRESH_BINARY)
    threshold_height, threshold_width = threshold_eye.shape
    left_side_threshold = threshold_eye[0:threshold_height, 0:int(threshold_width/2)]
    left_side_white = cv2.countNonZero(left_side_threshold)

    right_side_threshold = threshold_eye[0:threshold_height, int(threshold_width/2):threshold_width]
    right_side_white = cv2.countNonZero(right_side_threshold)
    
    if left_side_white==0:
        gaze_ratio = 1
    elif right_side_white==0:
        gaze_ratio = 5
    else:
        gaze_ratio = left_side_white/right_side_white
    
    return gaze_ratio

#Frames Counter - Since videos work on frames
frames = 0
letter_index = 0

while(True):
    _ , frame = cap.read()
    frame = cv2.resize(frame, None, fx=0.5,fy=0.5)
    keyboard[:] = (0,0,0)
    frames += 1
    new_frame = np.zeros((500,500,3), np.uint8)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
#         face detection
#         x,y = face.left(),face.top()
#         x1,y1 = face.right(),face.bottom()
#         cv2.rectangle(frame, (x,y),(x1,y1),(0,255,0),2)

        landmarks = predictor(gray,face)
#         print(landmarks.part(36)) # Position of point 36 of face landmarks
#         x = landmarks.part(36).x
#         y = landmarks.part(36).y
#         cv2.circle(frame, (x,y),2,(0,0,255),2)
        
#         Detect Blinking
        left_eye_ratio = get_blinking_ratio([36,37,38,39,40,41], landmarks)
        right_eye_ratio = get_blinking_ratio([42,43,44,45,46,47], landmarks)
        
        blinking_ratio = (left_eye_ratio + right_eye_ratio)/2
        
        if(blinking_ratio>4.95):
            cv2.putText(frame, "BLINKING" , (50,150), font, 3, (255,0,0))
            
        #Three directions covered - Left side, Right side and Center
        #Gaze Detection
        gaze_ratio_left_eye = get_gaze_ratio([36,37,38,39,40,41], landmarks)
        gaze_ratio_right_eye = get_gaze_ratio([42,43,44,45,46,47], landmarks)
        
        gaze_ratio = (gaze_ratio_left_eye+gaze_ratio_right_eye)/2
        print(gaze_ratio)
        
#         cv2.putText(frame, str(left_side_white), (50,100), font, 2, (0,0,255), 3)
#         cv2.putText(frame, str(right_side_white), (50,150), font, 2, (0,0,255), 3)
        
        
        if gaze_ratio<=0.51:
            cv2.putText(frame, "RIGHT", (50,150), font, 2, (0,0,255), 3)
            new_frame[:] = (0,0,255)
        elif 0.51< gaze_ratio < 2.1:
            cv2.putText(frame, "CENTER", (50,150), font, 2, (0,0,255), 3)
        else:
            cv2.putText(frame, "LEFT", (50,150), font, 2, (0,0,255), 3)
            new_frame[:] = (255,0,0)
            
            
#         Showing detection
        
        
#         threshold_eye = cv2.resize(threshold_eye, None , fx=5, fy=5)
#         eye = cv2.resize(gray_eye, None , fx=5, fy=5)
# #         cv2.imshow("Eye",eye)
#         cv2.imshow("Threshold", threshold_eye)
# #         cv2.imshow("Mask",mask)
# #         cv2.imshow("Left eye",left_eye)
#         cv2.imshow("left",left_side_threshold)
#         cv2.imshow("right",right_side_threshold)
        
    #Letters
    if frames==10:
        letter_index += 1
        frames=0
        if letter_index==15:
            letter_index = 0
    
    for i in range(15): 
        if i==letter_index:
            light = True   
        else:
            light = False
        letter(i,keys_set_1[i],light)
    
    cv2.imshow("Frame",frame)
    cv2.imshow("New Frame",new_frame)
    cv2.imshow("Virtual keyboard",keyboard)
    
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

3.0
3.0
3.0
3.0
1.6
1.6
1.6
1.2222222222222223
1.2222222222222223
0.49375
0.49375
0.5923076923076923
0.6900452488687783
0.6900452488687783
0.726890756302521
0.625
0.625
0.6826923076923077
0.7333333333333334
0.7333333333333334
0.6428571428571428
0.6029411764705883
0.6029411764705883
0.65
0.65
0.9166666666666667
0.9166666666666667
0.7047619047619047
0.7047619047619047
3.5666666666666664
3.5666666666666664
4.541666666666666
4.607142857142858
4.607142857142858
3.5
3.5
3.4583333333333335
3.4583333333333335
3.0
3.0
2.4214285714285717
2.4214285714285717
1.7451923076923077
1.08125
1.08125
1.1722222222222223
1.15625
1.15625
1.125
1.125
2.2083333333333335
2.625
2.625
2.615079365079365
2.615079365079365
2.1625
2.4005847953216373
2.4005847953216373
1.1904761904761905
1.0277777777777777
1.0277777777777777
1.1875
1.1875
0.9500000000000001
0.9500000000000001
1.0535714285714286
1.0535714285714286
0.9722222222222223
0.9722222222222223
0.9025974025974026
0.86875
0.86875
0.9267676767676767
0.926767676767